In [3]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

In [145]:
import pandas as pd

df = pd.read_csv('data/final_results_sentiment.csv')
# df = dataset.tail(10).copy()
# df

In [146]:
df = df.rename(columns={
    '상호명': 'store_name',
    '주소': 'address',
    '작성자': 'writer',
    '평점들': 'ratings',
    '방문일': 'visit_date',
    '리뷰내용': 'review_content',
    '태그들': 'tags'
})

In [103]:
df

,store_name,address,writer,ratings,visit_date,review_content,tags,label,score,confidence
0,Buto,서울 용산구 한남대로27가길 32,우아한 장금씨,리뷰 838,방문일 | 10.5.토 | 2024년 10월 5일 토요일 | 1번째 방문 | 인증 ...,흑백요리사 셀럽의 셰프 임희원 셰프님의 아늑한 분위기 모던 한식 다이닝 바 부토예요...,음식이 맛있어요 | 특별한 메뉴가 있어요 | 재료가 신선해요 | 아늑해요 | 친절해요,1,NaN,0.9954
1,Buto,서울 용산구 한남대로27가길 32,lov****,리뷰 497,방문일 | 10.4.금 | 2024년 10월 4일 금요일 | 1번째 방문 | 인증 ...,흑백요리사 출연하신 셰프님의 베지테리언 사시미를 보고 꼭 먹어보고 싶다는 생각이 간...,음식이 맛있어요 | 재료가 신선해요 | 아늑해요 | 컨셉이 독특해요 | 친절해요,1,NaN,0.9954
2,Buto,서울 용산구 한남대로27가길 32,최태욱82,리뷰 15,방문일 | 9.29.일 | 2024년 9월 29일 일요일 | 1번째 방문 | 인증 ...,좋아요,음식이 맛있어요 | 기본 안주가 좋아요 | 술이 다양해요 | 특별한 메뉴가 있어요,1,NaN,0.9950
3,Buto,서울 용산구 한남대로27가길 32,latte14,"리뷰 1,227",방문일 | 23.12.16.토 | 2023년 12월 16일 토요일 | 1번째 방문 ...,여러가지 다 특이하고 맛있는 음식들. 가격대에 비해 양은 적지만 연말 분위기에 와인...,음식이 맛있어요,1,NaN,0.9952
4,Buto,서울 용산구 한남대로27가길 32,예진엄마58,리뷰 60,방문일 | 1.6.토 | 2024년 1월 6일 토요일 | 1번째 방문 | 인증 수단...,리뷰들이 다 좋아서 기대했는데 메뉴가 특이하긴 하지만 조금씩 부족한 맛이라 실망스러...,차분한 분위기에요,0,NaN,0.0160
5,Buto,서울 용산구 한남대로27가길 32,nij****,리뷰 5,방문일 | 23.10.17.화 | 2023년 10월 17일 화요일 | 1번째 방문 ...,"다양하게 주문해서 먹었는데, 음식들이 모두 맛있고 특색 있습니다.\n와인과도 잘 어...",오래 머무르기 좋아요 | 인테리어가 멋져요 | 대화하기 좋아요 | 음식이 맛있어요 ...,1,NaN,0.9960
6,Buto,서울 용산구 한남대로27가길 32,OㅅOmg,리뷰 915,방문일 | 23.5.27.토 | 2023년 5월 27일 토요일 | 1번째 방문 | ...,분위기도 좋고 음식도 맛있었어요ㅎㅎ,인테리어가 멋져요 | 대화하기 좋아요 | 음식이 맛있어요 | 기본 안주가 좋아요,1,NaN,0.9945
7,Buto,서울 용산구 한남대로27가길 32,Kkkkkkkkkkkkk,리뷰 788,방문일 | 23.11.11.토 | 2023년 11월 11일 토요일 | 1번째 방문 ...,굿,혼술하기 좋아요,1,NaN,0.9942
8,Buto,서울 용산구 한남대로27가길 32,퐝썬,리뷰 363,방문일 | 22.8.10.수 | 2022년 8월 10일 수요일 | 1번째 방문 | ...,기대없이 갔다가 구석진 좋은 자리에 너무 맛있어서 계속 주문...배터짐ㅋㅋㅋ,음식이 맛있어요,1,NaN,0.9945
9,Buto,서울 용산구 한남대로27가길 32,젠젠일상,리뷰 280,방문일 | 22.7.13.수 | 2022년 7월 13일 수요일 | 1번째 방문 | ...,분위기도 좋도 음식도 괜찮은데 가격이 좀 비싸요~ 와인은 무조건 시켜야하더라구요~!,기본 안주가 좋아요,1,NaN,0.9566


In [7]:
def prompts(example):
    prompt_list = []
    for i in range(len(example['instruction'])):
        prompt_list.append(
f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>{example['instruction'][i]}<|eot_id|><|start_header_id|>assistant<|end_header_id|>{example['output'][i]}<|eot_id|>"""
        )
    return prompt_list

In [8]:
# LoRA 설정 : 양자화된 모델에서 Adaptor를 붙여서 학습할 파라미터만 따로 구성함

BASE_MODEL = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

lora_config = LoraConfig(
    r=8,
    lora_alpha = 32,
    lora_dropout = 0.1,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL,device_map="auto")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [9]:
# 모델의 그래디언트 체크포인팅을 활성화하여 메모리 사용량 최적화
model.gradient_checkpointing_enable()

# 모델의 파라미터가 k-비트 양자화에 적합하도록 조정 준비
model = prepare_model_for_kbit_training(model)

# freezing
model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

trainable params: 20,971,520 || all params: 8,051,232,768 || trainable%: 0.2605


In [10]:
# base model tokenizer

# tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = 'right'

In [11]:
# # vLLM 이용한 inference
# from vllm import LLM, SamplingParams
# from transformers import AutoTokenizer, pipeline


# llm = LLM(model=BASE_MODEL)

# tokenizer = AutoTokenizer.from_pretrained(
#     model=BASE_MODEL,
#     gpu_memory_utilization=0.8,  # GPU 메모리 사용률 제한
#     max_model_len=2048,          # 최대 시퀀스 길이 제한
#     tensor_parallel_size=1       # 단일 GPU 사용
# )
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = 'right'



In [12]:
import os
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
import torch

# 메모리 정리
import gc
torch.cuda.empty_cache()  # GPU 메모리 정리
gc.collect()             # CPU 메모리 정리

# 환경 변수 설정으로 메모리 관리 개선
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'

# 모델 로드 방식 수정
model = AutoAWQForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="cpu",
    torch_dtype=torch.float32,
    low_cpu_mem_usage=True,
    offload_folder="temp_model_storage",  # 임시 저장소 사용
    offload_state_dict=True,  # 상태 딕셔너리 오프로드
)

# 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True,
    use_fast=False  # 더 안정적인 로딩을 위해
)

# 양자화 설정
quant_config = {
    "zero_point": True,
    "q_group_size": 128,
    "w_bit": 4,
    "version": "GEMM"
}

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [13]:
quant_path = 'models/llama-3-korean-awq'

# 메모리 관리
import gc
gc.collect()

# 양자화 실행
model.quantize(tokenizer, quant_config=quant_config)

# 성공하면 저장
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

Repo card metadata block was not found. Setting CardData to empty.
AWQ: 100%|██████████| 32/32 [28:30<00:00, 53.44s/it]


('models/llama-3-korean-awq/tokenizer_config.json',
 'models/llama-3-korean-awq/special_tokens_map.json',
 'models/llama-3-korean-awq/tokenizer.json')

In [17]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer, pipeline

# vLLM에서는 경로만 직접 지정
llm = LLM(
    model="models/llama-3-korean-awq",
    trust_remote_code=True,
    gpu_memory_utilization=0.8,  # GPU 메모리 사용량 제한
    max_model_len=2048           # 최대 시퀀스 길이 제한
)

# transformers는 repo_type 사용 가능
tokenizer = AutoTokenizer.from_pretrained(
    "models/llama-3-korean-awq",
    repo_type="local",
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

INFO 12-04 03:12:35 config.py:1861] Downcasting torch.float32 to torch.float16.


INFO 12-04 03:12:39 config.py:350] This model supports multiple tasks: {'generate', 'embedding'}. Defaulting to 'generate'.
INFO 12-04 03:12:39 awq_marlin.py:109] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
INFO 12-04 03:12:39 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='models/llama-3-korean-awq', speculative_config=None, tokenizer='models/llama-3-korean-awq', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, 

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 12-04 03:12:42 model_runner.py:1077] Loading model weights took 5.3452 GB
INFO 12-04 03:12:42 worker.py:232] Memory profiling results: total_gpu_memory=23.69GiB initial_memory_usage=6.14GiB peak_torch_memory=6.99GiB memory_usage_post_profile=6.15GiB non_torch_memory=0.33GiB kv_cache_size=11.62GiB gpu_memory_utilization=0.80
INFO 12-04 03:12:42 gpu_executor.py:113] # GPU blocks: 5951, # CPU blocks: 2048
INFO 12-04 03:12:42 gpu_executor.py:117] Maximum concurrency for 2048 tokens per request: 46.49x
INFO 12-04 03:12:48 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 12-04 03:12:48 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 1

In [92]:
# # 새로운 방식
# label = 0  # 부정적인 리뷰이므로 0
# text = '초밥만 먹을만하고;; 정말 많이 실망하고 갑니다…'


In [93]:
# instruction = '음식이 전반적으로 짠데, 특시 스테이크 머쉬룸 리조또는 믿기 어려울만큼 짰다.'

# messages = [
#     {
#       "role": "system",
#       "content": """당신은 한국어 문장의 감성 분석을 전문으로 하는 매우 유능한 AI 어시스턴트입니다. 제공된 각 한글 문장의 감정을 1점에서 10점까지, 0.5점 단위로 평가하는 것이 당신의 임무입니다.
# 1. 감정 점수 (1-10점, 0.5점 단위)
# 2. 신뢰도 점수 (0-100%, 1% 단위)

# - **평가 척도:**
#     - **10.0:** 매우 긍정적인 감정
#     - **9.5 – 10.0:** 매우 긍정적인 감정
#     - **9.0 – 9.5:** 긍정적인 감정
#     - **8.5 – 9.0:** 약간 긍정적인 감정
#     - **8.0 – 8.5:** 경미하게 긍정적인 감정
#     - **7.5 – 8.0:** 중립에서 약간 긍정적인 감정
#     - **7.0 – 7.5:** 중립적인 감정
#     - **6.5 – 7.0:** 중립에서 약간 부정적인 감정
#     - **6.0 – 6.5:** 경미하게 부정적인 감정
#     - **5.5 – 6.0:** 약간 부정적인 감정
#     - **5.0 – 5.5:** 부정적인 감정
#     - **4.5 – 5.0:** 매우 부정적인 감정
#     - **4.0 – 4.5:** 극히 부정적인 감정
#     - **1.0 – 4.0:** 심각하게 부정적인 감정

# - **부정적 감정 인식 강화:**
#     - **부정적 단어 및 표현:** "짜다", "실망", "불만족", "나쁘다", "별로", "최악", "실패", "싫다", "안 좋다" 등
#     - **부정적 문맥 파악:** 문장 전체의 맥락에서 부정적인 의미를 파악
#     - **복합 감정 처리:** 문장 내에 긍정적 및 부정적 감정이 함께 있을 경우, 부정적 감정을 우선적으로 반영

# - **예시:**
#     - 예시 1:
#         - 문장: "음식이 전반적으로 짰다."
#         - 감정 점수: 4.5
#         - 신뢰도: 95%
#     - 예시 2:
#         - 문장: "서비스는 좋았지만 음식이 별로였다."
#         - 감정 점수: 5.5
#         - 신뢰도: 90%
#     - 예시 3:
#         - 문장: "전체적으로 만족스러웠다."
#         - 감정 점수: 8.5
#         - 신뢰도: 95%

# **신뢰도 점수 기준:**
#     - 문맥이 명확하고 감정 표현이 뚜렷할수록 높은 신뢰도
#     - 애매모호한 표현이나 복합적인 감정이 섞여있을 경우 낮은 신뢰도
#     - 문장이 짧거나 불완전한 경우 낮은 신뢰도

# **답변 형식:**
# 감정 점수: [점수]
# 신뢰도: [백분율]%"""
#     },
#     {
#       "role": "user",
#       "content": instruction
#     }, 
# ]


In [115]:
import re

In [147]:
def process_review(idx, row):
    label = row['label']
    text = row['review_content']

    messages = [
        {
            "role": "system", 
            "content": """당신은 한국어 문장의 감성 분석을 전문으로 하는 매우 유능한 AI 어시스턴트입니다. 
먼저 입력된 긍/부정 레이블(0=부정, 1=긍정)을 반드시 확인한 후, 해당 레이블을 참고하여 문장의 전체적인 감정을 분석합니다.

**입력 형식:**
- 레이블: [0 또는 1]
- 문장: [분석할 텍스트]

**분석 단계:**
1. **레이블 확인:** 
    - 0 = 부정
    - 1 = 긍정
2. **감정 점수 산출:**
    - **레이블이 0 (부정)일 경우:** 감정 점수는 **1.0 - 5.0** 사이에서 산출
        - **1.0:** 극히 부정적 (예: "진짜 이런 ㅈ같은 음식점은 다시는 가지마삼.")
        - **1.5:** 매우 부정적
        - **2.0:** 심각하게 부정적
        - **2.5:** 매우 부정적
        - **3.0:** 부정적
        - **3.5:** 약간 부정적
        - **4.0:** 보통 부정적
        - **4.5:** 약간 부정적
        - **5.0:** 경미하게 부정적
    - **레이블이 1 (긍정)일 경우:** 감정 점수는 **5.5 - 10.0** 사이에서 산출
        - **5.5:** 약간 긍정적
        - **6.0:** 경미하게 긍정적
        - **6.5:** 중립에서 약간 긍정적
        - **7.0:** 중립적
        - **7.5:** 중립에서 약간 긍정적
        - **8.0:** 경미하게 긍정적
        - **8.5:** 약간 긍정적
        - **9.0:** 긍정적
        - **9.5:** 매우 긍정적
        - **10.0:** 매우 긍정적
    - 점수는 **0.5점 단위**로 산출
3. **신뢰도 점수 산출:** (0-100%, 1% 단위)
    - 문맥이 명확하고 감정 표현이 뚜렷할수록 높은 신뢰도
    - 애매모호한 표현이나 복합적인 감정이 섞여있을 경우 낮은 신뢰도
    - 문장이 짧거나 불완전한 경우 낮은 신뢰도

**평가 척도:**

- **부정적 감정 (레이블=0):**
    - **1.0:** 극히 부정적인 감정
    - **1.5 – 2.0:** 매우 부정적인 감정
    - **2.5 – 3.0:** 심각하게 부정적인 감정
    - **3.5 – 4.0:** 부정적인 감정
    - **4.5 – 5.0:** 약간 부정적인 감정

- **긍정적 감정 (레이블=1):**
    - **5.5 – 6.0:** 약간 긍정적인 감정
    - **6.5 – 7.0:** 중립에서 약간 긍정적인 감정
    - **7.5 – 8.0:** 경미하게 긍정적인 감정
    - **8.5 – 9.0:** 긍정적인 감정
    - **9.5 – 10.0:** 매우 긍정적인 감정

**부정적 감정 인식 강화:**
- **부정적 단어 및 표현:** "짜다", "실망", "불만족", "나쁘다", "별로", "최악", "실패", "싫다", "안 좋다", "ㅈ같다", "가지마삼" 등
- **부정적 문맥 파악:** 문장 전체의 맥락에서 부정적인 의미를 파악
- **복합 감정 처리:** 문장 내에 긍정적 및 부정적 감정이 함께 있을 경우, 부정적 감정을 우선적으로 반영

**예시:**
- **예시 1 (레이블=0):**
    - 문장: "진짜 이런 ㅈ같은 음식점은 다시는 가지마삼."
    - 감정 점수: 1.0
    - 신뢰도: 98%
    - 평가: 극히 부정적인 감정
- **예시 2 (레이블=0):**
    - 문장: "음식이 전반적으로 짰다."
    - 감정 점수: 4.5
    - 신뢰도: 95%
    - 평가: 약간 부정적인 감정
- **예시 3 (레이블=1):**
    - 문장: "전체적으로 만족스러웠다."
    - 감정 점수: 8.5
    - 신뢰도: 95%
    - 평가: 약간 긍정적인 감정
  **예시 4 (레이블=1):**
    - 문장: "분위기도 좋도 음식도 괜찮은데 가격이 좀 비싸요~ 와인은 무조건 시켜야하더라구요~!"
    - 감정 점수: 7
    - 신뢰도: 95%
    - 평가: 중립에서 약간 긍정적인 감정

**답변 형식:**
- 감정 점수: [점수]
- 신뢰도: [백분율]%
- 평가: [감정 평가]

**주의 사항:**
- 레이블과 감정 점수가 일치하도록 반드시 확인
- 레이블=0일 경우, 감정 점수는 1.0에서 5.0 사이
- 레이블=1일 경우, 감정 점수는 5.5에서 10.0 사이
- 극단적인 감정 표현은 해당 범위의 최솟값 또는 최댓값에 가깝게 매겨지도록 반영"""
        },
        {
            "role": "user",
            "content": f"레이블: {label}\n문장: {text}"
        }, 
    ]

    prompt_message = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True,
    )

    # 결과 텍스트에서 감정 점수 추출
    eos_token_id = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

    outputs = llm.generate(prompt_message, SamplingParams(stop_token_ids=eos_token_id, temperature=0.1, top_p=0.95,max_tokens=2048))

    for output in outputs:
        generated_text = output.outputs[0].text
        score_match = re.search(r'감정 점수: (\d+\.?\d*)', generated_text)
        if score_match:
            return float(score_match.group(1))
    return None


In [148]:
# 4. 각 리뷰 처리 및 score 계산
for idx, row in df.iterrows():
    df.at[idx, 'score'] = process_review(idx,row)
    
    # 10개마다 중간 저장 (선택사항)
    if idx % 1000 == 0 and idx > 0:
        df.to_csv('data/reviews_with_scores_backup.csv', index=False, encoding='utf-8-sig')
        print(f"Backup saved at row {idx}")

# CSV 파일로 저장
output_path = 'data/reviews_with_scores.csv'
df.to_csv(output_path, index=False, encoding='utf-8-sig')  # utf-8-sig로 한글 깨짐 방지
print(f"\nResults saved to: {output_path}")














Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.65s/it, est. speed input: 1041.86 toks/s, output: 87.93 toks/s]












Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.98s/it, est. speed input: 823.27 toks/s, output: 95.69 toks/s]












Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.21it/s, est. speed input: 1731.16 toks/s, output: 71.73 toks/s]












Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.24s/it, est. speed input: 1174.05 toks/s, output: 86.97 toks/s]












Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.64s/it, est. speed input: 897.17 toks/s, output: 93.99 toks/s]












Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.33s/it, est. speed input: 1103.92 toks/s, output: 88.92 toks/s]












Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.33s/it, est. speed input: 1084.06 toks/s, output: 89.08 toks/s]












Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.13

Backup saved at row 1000


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.62s/it, est. speed input: 897.15 toks/s, output: 92.55 toks/s]


Backup saved at row 2000


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.76s/it, est. speed input: 825.25 toks/s, output: 94.72 toks/s]


...

Backup saved at row 25000


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.17it/s, est. speed input: 1667.61 toks/s, output: 71.43 toks/s]


Backup saved at row 26000


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.38s/it, est. speed input: 1047.12 toks/s, output: 88.28 toks/s]


Results saved to: data/reviews_with_scores.csv


In [143]:
# for idx, row in df.iterrows():
#     df.at[idx, 'score'] = process_review(idx,row)
#     # 현재 처리된 행만 출력
#     print(f"\nReview {idx}:")
#     print(f"Content: {row['review_content']}")
#     print(f"Label: {row['label']}")
#     print(f"Score: {df.at[idx, 'score']}")
#     print("-" * 50)  # 구분선 출력













Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.21s/it, est. speed input: 1185.40 toks/s, output: 81.35 toks/s]



Review 26384:
Content: 빵 맛있어요
Label: 1
Score: 9.0
--------------------------------------------------














Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.47s/it, est. speed input: 974.78 toks/s, output: 91.28 toks/s]



Review 26385:
Content: 특색있는 빵 때문에 방문해요
Label: 1
Score: 8.0
--------------------------------------------------














Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.33s/it, est. speed input: 1085.37 toks/s, output: 88.88 toks/s]


...


Review 26391:
Content: 11시쯤 방문하니 손님이 매우 많았어요. 매장내부가 좁아서 문 열자마자 쟁반부터 집어야 하실듯 ㅎㅎ
저는 매장에서 빵 먹고 나왔고요.
오전에 가시면 매장 곳곳에 빵이 놓여있는걸 보시게 됩니다 ㅎ
식빵도 사봤는데, 우유맛이 강하지 않은 매우 담백한 식빵이에요.
가볍게 구워서 잼 발라도 맛있고, 계란후라이 하나 얹어서 먹어도 조화로운 하얀 도화지 같은 맛!
Label: 1
Score: 8.0
--------------------------------------------------














Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.23s/it, est. speed input: 1291.27 toks/s, output: 82.13 toks/s]



Review 26392:
Content: 메론빵, 크로아상, 바게트, 새우카츠 버거

오랜만에 방문한 아오이토리. 왜이렇게 오랜만에 가게 됐을까요! 빵이 이렇게나 맛있는데.. 다 맛있었지만! 버터풍미 가득하고 부드러웠던 크로아상과 이건 보이면 꼭 사와야겠다!! 했던 새우카츠버거.. 남편이 내것도 골라올걸.. 하고 후회하더라구요 ㅋㅋ 느끼함 없이 오히려 담백한 편이고 맛있고 든든합니다! 벌써 그리워서 아오이토리 언제 또 갈 수 있으려나 기대되네요 *_*
Label: 1
Score: 8.5
--------------------------------------------------














Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.82s/it, est. speed input: 798.22 toks/s, output: 95.92 toks/s]


Review 26393:
Content: 빵이 가격대가 좀 있는편이긴 하지만
다 너무 맛있어서 자꾸 생각나요 ㅎㅎ
Label: 1
Score: 8.5
--------------------------------------------------


In [110]:
# messages = [
#     {
#         "role": "system",
#         "content": """당신은 한국어 문장의 감성 분석을 전문으로 하는 매우 유능한 AI 어시스턴트입니다. 
# 먼저 입력된 긍/부정 레이블(0=부정, 1=긍정)을 반드시 확인한 후, 해당 레이블을 참고하여 문장의 전체적인 감정을 분석합니다.

# **입력 형식:**
# - 레이블: [0 또는 1]
# - 문장: [분석할 텍스트]

# **분석 단계:**
# 1. **레이블 확인:** 
#     - 0 = 부정
#     - 1 = 긍정
# 2. **감정 점수 산출:**
#     - **레이블이 0 (부정)일 경우:** 감정 점수는 **1.0 - 5.0** 사이에서 산출
#         - **1.0:** 극히 부정적 (예: "진짜 이런 ㅈ같은 음식점은 다시는 가지마삼.")
#         - **1.5:** 매우 부정적
#         - **2.0:** 심각하게 부정적
#         - **2.5:** 매우 부정적
#         - **3.0:** 부정적
#         - **3.5:** 약간 부정적
#         - **4.0:** 보통 부정적
#         - **4.5:** 약간 부정적
#         - **5.0:** 경미하게 부정적
#     - **레이블이 1 (긍정)일 경우:** 감정 점수는 **5.5 - 10.0** 사이에서 산출
#         - **5.5:** 약간 긍정적
#         - **6.0:** 경미하게 긍정적
#         - **6.5:** 중립에서 약간 긍정적
#         - **7.0:** 중립적
#         - **7.5:** 중립에서 약간 긍정적
#         - **8.0:** 경미하게 긍정적
#         - **8.5:** 약간 긍정적
#         - **9.0:** 긍정적
#         - **9.5:** 매우 긍정적
#         - **10.0:** 매우 긍정적
#     - 점수는 **0.5점 단위**로 산출
# 3. **신뢰도 점수 산출:** (0-100%, 1% 단위)
#     - 문맥이 명확하고 감정 표현이 뚜렷할수록 높은 신뢰도
#     - 애매모호한 표현이나 복합적인 감정이 섞여있을 경우 낮은 신뢰도
#     - 문장이 짧거나 불완전한 경우 낮은 신뢰도

# **평가 척도:**

# - **부정적 감정 (레이블=0):**
#     - **1.0:** 극히 부정적인 감정
#     - **1.5 – 2.0:** 매우 부정적인 감정
#     - **2.5 – 3.0:** 심각하게 부정적인 감정
#     - **3.5 – 4.0:** 부정적인 감정
#     - **4.5 – 5.0:** 약간 부정적인 감정

# - **긍정적 감정 (레이블=1):**
#     - **5.5 – 6.0:** 약간 긍정적인 감정
#     - **6.5 – 7.0:** 중립에서 약간 긍정적인 감정
#     - **7.5 – 8.0:** 경미하게 긍정적인 감정
#     - **8.5 – 9.0:** 긍정적인 감정
#     - **9.5 – 10.0:** 매우 긍정적인 감정

# **부정적 감정 인식 강화:**
# - **부정적 단어 및 표현:** "짜다", "실망", "불만족", "나쁘다", "별로", "최악", "실패", "싫다", "안 좋다", "ㅈ같다", "가지마삼" 등
# - **부정적 문맥 파악:** 문장 전체의 맥락에서 부정적인 의미를 파악
# - **복합 감정 처리:** 문장 내에 긍정적 및 부정적 감정이 함께 있을 경우, 부정적 감정을 우선적으로 반영

# **예시:**
# - **예시 1 (레이블=0):**
#     - 문장: "진짜 이런 ㅈ같은 음식점은 다시는 가지마삼."
#     - 감정 점수: 1.0
#     - 신뢰도: 98%
#     - 평가: 극히 부정적인 감정
# - **예시 2 (레이블=0):**
#     - 문장: "음식이 전반적으로 짰다."
#     - 감정 점수: 4.5
#     - 신뢰도: 95%
#     - 평가: 약간 부정적인 감정
# - **예시 3 (레이블=1):**
#     - 문장: "전체적으로 만족스러웠다."
#     - 감정 점수: 8.5
#     - 신뢰도: 95%
#     - 평가: 약간 긍정적인 감정

# **답변 형식:**
# - 감정 점수: [점수]
# - 신뢰도: [백분율]%
# - 평가: [감정 평가]

# **주의 사항:**
# - 레이블과 감정 점수가 일치하도록 반드시 확인
# - 레이블=0일 경우, 감정 점수는 1.0에서 5.0 사이
# - 레이블=1일 경우, 감정 점수는 5.5에서 10.0 사이
# - 극단적인 감정 표현은 해당 범위의 최솟값 또는 최댓값에 가깝게 매겨지도록 반영
# """
#     },
#     {
#         "role": "user",
#         "content": f"레이블: {label}\n문장: {text}"
#     }, 
# ]

In [111]:
# messages = [
#     {
#         "role": "system",
#         "content": """당신은 한국어 문장의 감성 분석을 전문으로 하는 매우 유능한 AI 어시스턴트입니다. 
# 먼저 입력된 긍/부정 레이블(0=부정, 1=긍정)을 반드시 확인한 후, 해당 레이블을 참고하여 문장의 전체적인 감정을 분석합니다.

# **입력 형식:**
# - 레이블: [0 또는 1]
# - 문장: [분석할 텍스트]

# **분석 단계:**
# 1. **레이블 확인:** 
#     - 0 = 부정
#     - 1 = 긍정
# 2. **감정 점수 산출:**
#     - **레이블이 0 (부정)일 경우:** 감정 점수는 **1.0 - 5.0** 사이에서 산출
#         - 1.0: 극히 부정적
#         - 5.0: 약간 부정적
#     - **레이블이 1 (긍정)일 경우:** 감정 점수는 **5.5 - 10.0** 사이에서 산출
#         - 5.5: 약간 긍정적
#         - 10.0: 매우 긍정적
#     - 점수는 **0.5점 단위**로 산출
# 3. **신뢰도 점수 산출:** (0-100%, 1% 단위)
#     - 문맥이 명확하고 감정 표현이 뚜렷할수록 높은 신뢰도
#     - 애매모호한 표현이나 복합적인 감정이 섞여있을 경우 낮은 신뢰도
#     - 문장이 짧거나 불완전한 경우 낮은 신뢰도

# **평가 척도:**

# - **부정적 감정 (레이블=0):**
#     - **5.0:** 약간 부정적인 감정
#     - **4.5 – 5.0:** 약간 부정적인 감정
#     - **4.0 – 4.5:** 매우 부정적인 감정
#     - **1.0 – 4.0:** 심각하게 부정적인 감정

# - **긍정적 감정 (레이블=1):**
#     - **10.0:** 매우 긍정적인 감정
#     - **9.5 – 10.0:** 매우 긍정적인 감정
#     - **9.0 – 9.5:** 긍정적인 감정
#     - **8.5 – 9.0:** 약간 긍정적인 감정
#     - **8.0 – 8.5:** 경미하게 긍정적인 감정
#     - **7.5 – 8.0:** 중립에서 약간 긍정적인 감정
#     - **7.0 – 7.5:** 중립적인 감정
#     - **6.5 – 7.0:** 중립에서 약간 부정적인 감정
#     - **6.0 – 6.5:** 경미하게 부정적인 감정
#     - **5.5 – 6.0:** 약간 부정적인 감정

# **부정적 감정 인식 강화:**
# - **부정적 단어 및 표현:** "짜다", "실망", "불만족", "나쁘다", "별로", "최악", "실패", "싫다", "안 좋다" 등
# - **부정적 문맥 파악:** 문장 전체의 맥락에서 부정적인 의미를 파악
# - **복합 감정 처리:** 문장 내에 긍정적 및 부정적 감정이 함께 있을 경우, 부정적 감정을 우선적으로 반영

# **답변 형식:**
# - 감정 점수: [점수]
# - 신뢰도: [백분율]%
# - 평가: [감정 평가]

# **주의 사항:**
# - 레이블과 감정 점수가 일치하도록 반드시 확인
# - 레이블=0일 경우, 감정 점수는 1.0에서 5.0 사이
# - 레이블=1일 경우, 감정 점수는 5.5에서 10.0 사이"""
#     },
#     {
#         "role": "user",
#         "content": f"레이블: {label}\n문장: {text}"
#     }, 
# ]

In [112]:
# prompt_message = tokenizer.apply_chat_template(
#         messages, 
#         tokenize=False, 
#         add_generation_prompt=True,
# )

# # 결과 텍스트에서 감정 점수 추출
# eos_token_id = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

# outputs = llm.generate(prompt_message, SamplingParams(stop_token_ids=eos_token_id, temperature=0.1, top_p=0.95,max_tokens=2048))



Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.63s/it, est. speed input: 858.54 toks/s, output: 91.50 toks/s]


In [113]:


# for output in outputs:
#     generated_text = output.outputs[0].text
#     # 정규표현식으로 감정 점수 추출
#     import re
#     score_match = re.search(r'감정 점수: (\d+\.?\d*)', generated_text)
#     if score_match:
#         score = float(score_match.group(1))
#         df.at[idx, 'score'] = score

# # 진행상황 출력
# if idx % 100 == 0:
#     print(f"Processed {idx} rows")

# # 결과 확인
# print(df[['review_content', 'label', 'score']].head())

                                      review_content  label  score
0  흑백요리사 셀럽의 셰프 임희원 셰프님의 아늑한 분위기 모던 한식 다이닝 바 부토예요...      1    NaN
1  흑백요리사 출연하신 셰프님의 베지테리언 사시미를 보고 꼭 먹어보고 싶다는 생각이 간...      1    NaN
2                                                좋아요      1    NaN
3  여러가지 다 특이하고 맛있는 음식들. 가격대에 비해 양은 적지만 연말 분위기에 와인...      1    NaN
4  리뷰들이 다 좋아서 기대했는데 메뉴가 특이하긴 하지만 조금씩 부족한 맛이라 실망스러...      0    NaN


In [97]:
# eos_token_id = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

# outputs = llm.generate(prompt_message, SamplingParams(stop_token_ids=eos_token_id, temperature=0.1, top_p=0.95,max_tokens=2048))

# for output in outputs:
#     propt = output.prompt
#     generated_text = output.outputs[0].text
#     print(generated_text)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.31s/it, est. speed input: 1032.73 toks/s, output: 87.97 toks/s]

**감정 점수: 3.0**
**신뢰도: 98%**
**평가: 부정적인 감정**

이 문장은 "초밥만 먹을만하고"라는 표현으로 시작하여 부정적인 감정을 전달하고, "실망"이라는 단어를 사용하여 강한 부정적 감정을 강조합니다. 또한 "갑니다"라는 표현으로 문장의 끝을 부정적으로 마무리합니다. 따라서, 이 문장은 부정적인 감정으로 평가됩니다.


In [11]:
# instruction = '파이썬 merge sort 코드를 제시해주고 자세한 설명을 해줘'

# messages = [
#     {
#       "role": "system",
#       "content": "당신은 훌륭한 AI 코드 어시스턴트입니다. 짧은 답변을 제시하고, 다음으로 상세 설명을 해주세요. You are a great AI code assistant. Give a short answer, then elaborate. "
#     },
#     {
#       "role": "user",
#       "content": instruction
#     }, 
# ]


In [12]:

# prompt_message = tokenizer.apply_chat_template(
#         messages, 
#         tokenize=False, 
#         add_generation_prompt=True,
# )


In [ ]:
# eos_token_id = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

# outputs = llm.generate(prompt_message, SamplingParams(stop_token_ids=eos_token_id, temperature=0.8, top_p=0.95,max_tokens=512))
